## 需要注意一个重点：pad处理时需要把test和train的长度pad到一致

In [1]:
import pandas as pd
import numpy as np
from torch import nn
from gensim.models.word2vec import Word2Vec
import jieba
from torch.nn.utils.rnn import pad_sequence,pad_packed_sequence,pack_sequence,pack_padded_sequence
from torch.nn import CrossEntropyLoss
from transformers import get_scheduler

In [2]:
train_data=pd.read_csv("/data/linqinhong/kaggle/yitushibie/data/yitushibie/train.csv",header=None,names=["text","title"],sep="\t")
test_data=pd.read_csv("/data/linqinhong/kaggle/yitushibie/data/yitushibie/test.csv",header=None,names=["text","title"],sep="\t")

In [3]:
train_data["process"]=train_data["text"].apply(jieba.lcut)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.601 seconds.
Prefix dict has been built successfully.


In [4]:
test_data["process"]=test_data["text"].apply(jieba.lcut)

In [5]:
from nltk.corpus import stopwords
stopword=stopwords.words("chinese")

In [6]:
train_data["process"]=train_data["process"].apply(lambda x: [i for i in x if i not in stopword])

In [7]:
test_data["process"]=test_data["process"].apply(lambda x: [i for i in x if i not in stopword])

In [8]:
process=list(train_data["process"])+list(test_data["process"])

In [9]:
CBOW=Word2Vec(sentences=process,sg=0,min_count=1,window=4,vector_size=300,epochs=300,workers=10)

In [10]:
def get_num(text):
    total_num = []
    if text == []:
        return torch.tensor([])
    for word in text:
        index = CBOW.wv.key_to_index[word]
        total_num.append(index)
    return torch.tensor(total_num,dtype=torch.int)

In [11]:
def get_vec(text):
    total_vec=[]
    if text==[]:
        return None
    for word in text:
        index=CBOW.wv.key_to_index[word]
        vec=np.reshape(CBOW.wv[index],(1,300))
        #vec=(CBOW.wv[index])
        total_vec.append(vec)
    return np.concatenate(total_vec,axis=0)

In [12]:
import torch

In [13]:
train_data["num"]=train_data["process"].apply(get_num)

In [14]:
test_data["num"]=test_data["process"].apply(get_num)

In [15]:
pad_num=len(CBOW.wv.key_to_index)

In [16]:
num_list=list(train_data["num"])

In [48]:
test_num_list=list(test_data["num"])

In [69]:
test_num_list[0]

tensor([   91,  2683,  3476, 13067, 13067, 13067, 13067, 13067, 13067, 13067,
        13067, 13067, 13067, 13067, 13067, 13067, 13067, 13067, 13067, 13067,
        13067, 13067], dtype=torch.int32)

In [70]:
padded_numtensor=pad_sequence(num_list,padding_value=pad_num,batch_first=True)

In [71]:
test_pad_num=pad_sequence(test_num_list,padding_value=pad_num,batch_first=True)

In [72]:
len(test_pad_num[0])

22

In [60]:
padded_numlist=[i.tolist() for i in padded_numtensor]

In [73]:
test_data["padded_num"]=[i.tolist() for i in test_pad_num]

In [19]:
train_data["padded_num"]=padded_numlist

In [20]:
train_data.drop("num",inplace=True,axis=1)

In [21]:
CBOW.wv.vectors.shape

(13067, 300)

In [22]:
embedding=torch.tensor(np.concatenate((CBOW.wv.vectors,np.reshape(np.array([0.0] * 300),(1,300))),axis=0),dtype=torch.double)
"""embedding=np.concatenate((CBOW.wv.vectors,np.reshape(np.array([0.0] * 300),(1,300))),axis=0)"""

'embedding=np.concatenate((CBOW.wv.vectors,np.reshape(np.array([0.0] * 300),(1,300))),axis=0)'

In [23]:
def get_tensor(text):
    tensor=[]
    for i in text:
        tensor.append(torch.reshape(embedding[i],(1,-1)))
    return torch.cat(tensor,dim=0)
"""def get_tensor(text):
    nplist=[]
    for i in text:
        nplist.append(np.reshape(embedding[i],(1,-1)))
    return np.concatenate(nplist,axis=1)"""

'def get_tensor(text):\n    nplist=[]\n    for i in text:\n        nplist.append(np.reshape(embedding[i],(1,-1)))\n    return np.concatenate(nplist,axis=1)'

In [24]:
train_data["vec"]=train_data["padded_num"].apply(get_tensor)

In [25]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
"""class mydataset(Dataset):
    def __init__(self,data,label):
        self.data = data
        self.label = label
        self.len = len(self.label)

    def __getitem__(self, index):
        return self.data[index], self.label[index]

    def __len__(self):
        return self.len"""
#train_set=Dataset(train_data[["vec","label"]])


In [26]:
labelsset=set(train_data["title"])
labelsdict={}
reverser_dict={}
for index,keys in enumerate(labelsset):
    labelsdict[keys]=index
for index,keys in enumerate(labelsset):
    reverser_dict[index]=keys

In [27]:
def convert(text):
    return labelsdict[text]

In [28]:
train_data["label"]=train_data["title"].apply(convert)

In [30]:
train_data.drop(["text","title","process"],axis=1,inplace=True)

In [40]:
device=torch.device("cuda:1") if torch.cuda.is_available()==True else torch.device("cpu")
from torch import relu
class LSTMModule(nn.Module):
    def __init__(self,input_channels,hidden_channels,num_layers,output_channels,embedding):
        super(LSTMModule,self).__init__()
        self.num_layers=num_layers
        self.input=input_channels
        self.hidden=hidden_channels
        self.output=output_channels
        self.lstm=nn.LSTM(input_channels,hidden_channels,num_layers,batch_first=True,dropout=0.3)
        self.fc=nn.Linear(hidden_channels,hidden_channels)
        self.fc1=nn.Linear(hidden_channels,output_channels)
        #self.Embed=nn.Linear(embedding.shape[0],embedding.shape[1],)
        self.Embed=nn.Embedding(embedding.shape[0],embedding.shape[1],_weight=embedding)
        for para in self.Embed.parameters():
           para.requires_grad=False
        self.drop=nn.Dropout(0.5)
        self.batch=nn.modules.batchnorm.BatchNorm1d(22)

    def forward(self,x):
        x=self.Embed(x).float()
        batch_size,seq_len=x.shape[0],x.shape[1]
        h0=torch.zeros(self.num_layers,batch_size,self.hidden,dtype=torch.float,requires_grad=True).to(device)
        c0=torch.zeros(self.num_layers,batch_size,self.hidden,dtype=torch.float,requires_grad=True).to(device)
        out,(hn,cn)=self.lstm(x,(h0,c0))
        """out=self.fc1(out)[:,-1,:]"""
        out=self.fc(out)
        #print(out.shape)
        out=relu(out)
        self.batch(out)
        self.drop(out)
        #print(out.shape)
        out=self.fc1(out)
        #print(out.shape)
        out=torch.mean(out,dim=1)
        #print(out.shape)

        return out

In [33]:
from torch.utils.data import DataLoader,Dataset
class num_dataset(Dataset):
    def __init__(self,numlist,label):
        self.len=len(label)
        self.num=numlist
        self.label=label

    def __len__(self):
        return self.len

    def __getitem__(self,index):
        return self.num[index],self.label[index]


In [34]:
train_data["padded_tensor"]=train_data["padded_num"].apply(lambda x: torch.tensor(x))

In [74]:
test_data["padded_tensor"]=test_data["padded_num"].apply(lambda x: torch.tensor(x))

In [45]:
LSTM_class=LSTMModule(300,300,3,12,embedding)

In [46]:
train_set=num_dataset(train_data["padded_tensor"],train_data["label"])
train_loader=DataLoader(train_set,shuffle=True,batch_size=32)
"""train_set=num_dataset(train_data["padded_num",train_data["label"]])
train_loader=DataLoader(train_set,shuffle=True,batch_size=32)"""
loss_fn=CrossEntropyLoss()
epoches=100
total_loss=[]
total_acc=[]
num_step=epoches*len(train_loader)
import evaluate
from torch.optim.adam import Adam
metric=evaluate.load("accuracy")
#optimizer=Adam(LSTM_class.parameters(),lr=0.05)
#lr_scheduler=get_scheduler('linear',optimizer=optimizer,num_warmup_steps=num_step*0.2)
device=torch.device("cuda:1") if torch.cuda.is_available()==True else torch.device("cpu")
LSTM_class.to(device)
for epoch in range(epoches):
    loss_num=0
    acc=0
    optimizer=Adam(LSTM_class.parameters(),lr=0.005/(epoch/10+1))
    for i,(data,label) in enumerate(train_loader):
        #out=LSTM_class(data.float())
        #data.to(device)
        #label.to(device)
        out=LSTM_class(data.to(device))
        optimizer.zero_grad()
        loss=loss_fn(out,label.to(device))
        loss.backward()
        optimizer.step()
        #lr_scheduler.step()
        loss_num+=loss.item()
        metric.add_batch(predictions=torch.argmax(out,dim=1),references=label)
        acc+=metric.compute()["accuracy"]
    print("acc:",acc/len(train_loader))
    loss_num/=len(train_loader)
    total_loss.append(loss_num)
    print("loss:",loss_num)
    total_acc.append(acc/len(train_loader))


acc: 0.6662269129287599
loss: 0.9278104120477209
acc: 0.8771437994722955
loss: 0.4325538180856953
acc: 0.9046009234828496
loss: 0.32612642608730413
acc: 0.9234828496042217
loss: 0.26954858418255967
acc: 0.9296668865435356
loss: 0.23486853080573136
acc: 0.9444261213720316
loss: 0.19158058010989623
acc: 0.9492908970976254
loss: 0.16635953769088213
acc: 0.9572889182058048
loss: 0.14499924529475464
acc: 0.9608344327176781
loss: 0.1248739889810153
acc: 0.9656992084432717
loss: 0.10952890098779504
acc: 0.9686675461741425
loss: 0.09619220312297594
acc: 0.9731200527704486
loss: 0.08354377154454065
acc: 0.9769129287598944
loss: 0.07405599408510694
acc: 0.9781497361477572
loss: 0.07102931905709099
acc: 0.983344327176781
loss: 0.06267008564963551
acc: 0.9838390501319261
loss: 0.05200804310802632
acc: 0.9860653034300791
loss: 0.05653022466199014
acc: 0.9881266490765171
loss: 0.04085964136463096
acc: 0.9890336411609498
loss: 0.0415122119868693
acc: 0.9897757255936676
loss: 0.0399243076125363
acc: 0

In [53]:
from torch.utils.data import DataLoader,Dataset
class test_dataset(Dataset):
    def __init__(self,numlist):
        self.len=len(numlist)
        self.num=numlist


    def __len__(self):
        return self.len

    def __getitem__(self,index):
        return self.num[index]

In [81]:
test_set=test_dataset(test_data["padded_tensor"])
test_loader=DataLoader(test_set,shuffle=True,batch_size=32)
"""train_set=num_dataset(train_data["padded_num",train_data["label"]])
train_loader=DataLoader(train_set,shuffle=True,batch_size=32)"""
loss_fn=CrossEntropyLoss()
epoches=100
total_loss=[]
total_acc=[]
num_step=epoches*len(train_loader)
import evaluate
from torch.optim.adam import Adam
metric=evaluate.load("accuracy")
#optimizer=Adam(LSTM_class.parameters(),lr=0.05)
#lr_scheduler=get_scheduler('linear',optimizer=optimizer,num_warmup_steps=num_step*0.2)
device=torch.device("cuda:1") if torch.cuda.is_available()==True else torch.device("cpu")
LSTM_class.to(device)
total_pred=[]
for i,(data) in enumerate(test_loader):
    #out=LSTM_class(data.float())
    #data.to(device)
    #label.to(device)
    out=LSTM_class(data.to(device))
    pred=torch.argmax(out,dim=1)
    total_pred.extend(pred.tolist())

In [83]:
len(total_pred)

3000

In [84]:
test_rawdata=pd.read_csv("/data/linqinhong/kaggle/yitushibie/data/yitushibie/test.csv",header=None,names=["text","title"],sep="\t")

In [86]:
title=[reverser_dict[i] for i in total_pred]

In [89]:
test_rawdata["title"]=title

In [90]:
test_rawdata.to_csv("/data/linqinhong/kaggle/yitushibie/data/yitushibie/result_CBOW_LSTM.csv",index=None)